In [ ]:
# Engenharia de Dados 2
# Nauber Gois
# Dados: HR database

#Grupo: Jean Diniz , Paulo, Anizio

In [ ]:
!pip install pyspark

In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


In [ ]:
# prompt: quero importar https://github.com/JDinizfv/SparkLab todos os arquivos csv desse link

!git clone https://github.com/JDinizfv/SparkLab.git


fatal: destination path 'SparkLab' already exists and is not an empty directory.
mv: missing destination file operand after 'SparkLab/data/*.csv'
Try 'mv --help' for more information.


In [ ]:
# prompt: ler o arquivo amazon.csv no spark

df = spark.read.csv('hr_dashboard_data.csv', header=True, inferSchema=True)


In [ ]:
df.show(10)

+----------------+---+------+------------------+----------------+---------------------+--------------+----------+----------------+------------+------+
|            Name|Age|Gender|Projects Completed|Productivity (%)|Satisfaction Rate (%)|Feedback Score|Department|        Position|Joining Date|Salary|
+----------------+---+------+------------------+----------------+---------------------+--------------+----------+----------------+------------+------+
| Douglas Lindsey| 25|  Male|                11|              57|                   25|           4.7| Marketing|         Analyst|      Jan-20| 63596|
|Anthony Roberson| 59|Female|                19|              55|                   76|           2.8|        IT|         Manager|      Jan-99|112540|
|   Thomas Miller| 30|  Male|                 8|              87|                   10|           2.4|        IT|         Analyst|      Jan-17| 66292|
|    Joshua Lewis| 26|Female|                 1|              53|                    4|       

In [ ]:
# prompt: por favor renomear as colunas que contenham (%) como a Productivity (%) e Satisfaction Rate (%), quero que remova esses caracteres e strip()

df = df.withColumnRenamed('Productivity (%)', 'Productivity') \
       .withColumnRenamed('Satisfaction Rate (%)', 'Satisfaction')
df.show(10)


+----------------+---+------+------------------+------------+------------+--------------+----------+----------------+------------+------+
|            Name|Age|Gender|Projects Completed|Productivity|Satisfaction|Feedback Score|Department|        Position|Joining Date|Salary|
+----------------+---+------+------------------+------------+------------+--------------+----------+----------------+------------+------+
| Douglas Lindsey| 25|  Male|                11|          57|          25|           4.7| Marketing|         Analyst|      Jan-20| 63596|
|Anthony Roberson| 59|Female|                19|          55|          76|           2.8|        IT|         Manager|      Jan-99|112540|
|   Thomas Miller| 30|  Male|                 8|          87|          10|           2.4|        IT|         Analyst|      Jan-17| 66292|
|    Joshua Lewis| 26|Female|                 1|          53|           4|           1.4| Marketing|          Intern|      Jan-22| 38303|
|Stephanie Bailey| 43|  Male|     

In [ ]:
# prompt: Transformar o csv em parquet e fazer o spark lê-lo , chamar arquivo de hr_df

# Convert the CSV file to Parquet format
df.write.parquet('hr_df.parquet')

# Read the Parquet file into a new DataFrame
hr_df = spark.read.parquet('hr_df.parquet')

# Display the first 10 rows of the new DataFrame
hr_df.show(10)


+----------------+---+------+------------------+------------+------------+--------------+----------+----------------+------------+------+
|            Name|Age|Gender|Projects Completed|Productivity|Satisfaction|Feedback Score|Department|        Position|Joining Date|Salary|
+----------------+---+------+------------------+------------+------------+--------------+----------+----------------+------------+------+
| Douglas Lindsey| 25|  Male|                11|          57|          25|           4.7| Marketing|         Analyst|      Jan-20| 63596|
|Anthony Roberson| 59|Female|                19|          55|          76|           2.8|        IT|         Manager|      Jan-99|112540|
|   Thomas Miller| 30|  Male|                 8|          87|          10|           2.4|        IT|         Analyst|      Jan-17| 66292|
|    Joshua Lewis| 26|Female|                 1|          53|           4|           1.4| Marketing|          Intern|      Jan-22| 38303|
|Stephanie Bailey| 43|  Male|     

In [ ]:
# prompt: quero um df, nomeado df_avg agrupando por Department , Position, Gender, sem renomear colunas com a média (avg) das colunas Projects Completed, Productivity , Satisfaction, Feedback Score , Salary, Age

df_avg = hr_df.groupBy('Department', 'Position', 'Gender').avg("Productivity", "Satisfaction", "Projects Completed", "Feedback Score","Age","Salary")
df_avg.show()


+----------+----------------+------+------------------+------------------+-----------------------+-------------------+------------------+------------------+
|Department|        Position|Gender| avg(Productivity)| avg(Satisfaction)|avg(Projects Completed)|avg(Feedback Score)|          avg(Age)|       avg(Salary)|
+----------+----------------+------+------------------+------------------+-----------------------+-------------------+------------------+------------------+
| Marketing|       Team Lead|Female|              24.0|              20.8|                   14.4| 3.5800000000000005|              37.2|           96697.4|
|        IT|         Manager|  Male|              60.8|              43.2|                   21.8| 3.7400000000000007|              43.4|          109333.2|
|     Sales|         Manager|  Male|             28.75|              50.5|                   19.0| 2.8000000000000003|              48.0|          113529.5|
|     Sales|Junior Developer|  Male|              67.8|   

In [ ]:
df_gender = hr_df.groupBy("Gender").avg("Productivity", "Satisfaction", "Projects Completed", "Feedback Score")


In [ ]:
df_gender.show()

+------+-----------------+-----------------+-----------------------+-------------------+
|Gender|avg(Productivity)|avg(Satisfaction)|avg(Projects Completed)|avg(Feedback Score)|
+------+-----------------+-----------------+-----------------------+-------------------+
|Female|            42.97|            51.25|                  11.94|               3.01|
|  Male|            50.54|            48.62|                  10.97| 2.7560000000000002|
+------+-----------------+-----------------+-----------------------+-------------------+



In [ ]:
df_department = df.groupBy("Department").avg("Productivity", "Satisfaction", "Projects Completed", "Feedback Score")


In [ ]:
df_department.show()

+------+-----------------+-----------------+-----------------------+-------------------+
|Gender|avg(Productivity)|avg(Satisfaction)|avg(Projects Completed)|avg(Feedback Score)|
+------+-----------------+-----------------+-----------------------+-------------------+
|Female|            42.97|            51.25|                  11.94|               3.01|
|  Male|            50.54|            48.62|                  10.97| 2.7560000000000002|
+------+-----------------+-----------------+-----------------------+-------------------+



In [ ]:

df_position = df.groupBy("Position").avg("Productivity", "Satisfaction", "Projects Completed", "Feedback Score")
df_position.show()

+----------------+------------------+------------------+-----------------------+-------------------+
|        Position| avg(Productivity)| avg(Satisfaction)|avg(Projects Completed)|avg(Feedback Score)|
+----------------+------------------+------------------+-----------------------+-------------------+
|Junior Developer| 52.31428571428572| 54.05714285714286|      6.771428571428571| 2.8485714285714288|
|          Intern|41.333333333333336|55.266666666666666|                    2.7| 2.9533333333333327|
|Senior Developer|50.233333333333334|              50.4|     12.833333333333334|  2.633333333333333|
|         Analyst| 44.54545454545455| 39.78787878787879|      9.393939393939394| 2.7666666666666666|
|         Manager|            48.675|            55.575|                 19.975|  3.022500000000001|
|       Team Lead|            42.375|          43.40625|               14.96875|           3.034375|
+----------------+------------------+------------------+-----------------------+-----------

In [ ]:
# prompt: criar uma agrupamento por genero e media de salario , Age

df_gender_salary = hr_df.groupBy("Gender").avg("Salary", "Age")
df_gender_salary.show()


+------+-----------+--------+
|Gender|avg(Salary)|avg(Age)|
+------+-----------+--------+
|Female|   76985.22|   35.48|
|  Male|   76253.27|   33.82|
+------+-----------+--------+



In [ ]:
!mkdir datasink

In [ ]:
# prompt: armazene o df_avg, e particione por department e position , no formato parquet no diretorio datasink.

df_avg.write.partitionBy("Department", "Position").parquet("datasink/df_avg.parquet")


In [ ]:
# prompt: read e load do datasink no department = Finance

Finance_Analyst = spark.read \
    .format("parquet") \
    .load("/content/datasink/df_avg.parquet/Department=Finance/Position=Analyst/*.parquet")



In [ ]:
Finance_Analyst.show()

+------+-----------------+-----------------+-----------------------+-------------------+--------+-----------+
|Gender|avg(Productivity)|avg(Satisfaction)|avg(Projects Completed)|avg(Feedback Score)|avg(Age)|avg(Salary)|
+------+-----------------+-----------------+-----------------------+-------------------+--------+-----------+
|Female|             77.0|             44.0|                   10.0|                2.0|    28.0|    63299.0|
|  Male|             31.0|           42.125|                    8.0| 2.6750000000000003|  28.125|    68171.5|
+------+-----------------+-----------------+-----------------------+-------------------+--------+-----------+



In [ ]:

from pyspark.sql.functions import lit

In [ ]:
Finance_Analyst